In [139]:
import pandas as pd
import numpy as np
import os
import logging
import re

PATH_PUBLICATIONS = './'

In [140]:
# Create a custom logger
logger = logging.getLogger(__name__)

# Create handlers
c_handler = logging.StreamHandler()
f_handler = logging.FileHandler('file.log')

# Create formatters and add it to handlers
c_format = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
f_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
c_handler.setFormatter(c_format)
f_handler.setFormatter(f_format)

# Add handlers to the logger
logger.addHandler(c_handler)
logger.addHandler(f_handler)

# Web scrap comments 

In [ ]:
insta_link = "https://www.instagram.com/p/B_5BDxlDHzt/"

In [166]:
def get_image_code(insta_link):
    try:
        insta_image_code = insta_link.split('/')[-2]
    except:
        logger.error("get_image_code" + insta_image_code)
        
    return insta_image_code

In [168]:
def create_instaloader_file(insta_image_code):
    try:
        logger.info("building files for", insta_image_code)
        os.system(" instaloader --comments -- -" + insta_image_code)
    except:
        logger.error("create_instaloader" + insta_image_code)

In [169]:
def web_scrap_comments(insta_link):
    insta_image_code = get_image_code(insta_link)
    create_instaloader_file(insta_image_code)

In [170]:
web_scrap_comments("https://www.instagram.com/p/B_5BDxlDHzt/")

# Save comments

In [171]:
def clean_folders(PATH_PUBLICATIONS):
    for subdir, dirs, files in os.walk(PATH_PUBLICATIONS):
        # iterate over files in folders
        logger.info('Start cleaning folders')
        for file in files:
            file_path = os.path.join(subdir, file)
            # select comments file and the image
            if not file_path.endswith(('comments.json', '.jpg', '.ipynb', '.log')):
                os.remove(file_path)
    logger.info('End cleaning folders')

In [172]:
def save_comments(PATH_PUBLICATIONS):
    # iterate over all directories 
    logger.info("Start saving comments")
    for subdir, dirs, files in os.walk(PATH_PUBLICATIONS):
        # iterate over files in folders
        for file in files:
            file_path = os.path.join(subdir, file)
            if file_path.endswith('comments.json'):
                comments_df = pd.read_json(file_path)
                # keep only creation_at, text
                comments_df = comments_df[['created_at', 'text']].rename(columns={'created_at':'comment date',
                                                                                 'text':'comment'})
                comments_df.to_csv(subdir + "/comments.csv", index=False)
    logger.info("Comments saved")

In [173]:
# clean folders
clean_folders(PATH_PUBLICATIONS)

In [174]:
# save comments
save_comments(PATH_PUBLICATIONS)